In [1]:
sentence = ["In the sprawling landscape of human existence, where time flows ceaselessly like a river winding its way through the ages, we find ourselves caught in the currents of history, swept along by the tides of change and upheaval. From the dawn of civilization to the present day, we have grappled with the mysteries of the universe, striving to unlock the secrets of existence and uncover the meaning of our own existence. Along the way, we have built great empires and civilizations, forged mighty kingdoms and nations, and waged wars that have shaped the course of history. We have witnessed the rise and fall of countless dynasties, the triumphs and tragedies of human achievement, and the enduring resilience of the human spirit in the face of adversity. But for all our accomplishments, we are but fleeting specks in the vastness of time, mere blips on the cosmic radar of eternity. In the grand tapestry of existence, our lives are but a single thread, woven into the fabric of the universe with delicate precision. And yet, within the confines of our own mortality, we find the capacity for greatness, the ability to transcend our limitations and reach for the stars. We are creatures of both flesh and spirit, bound by the constraints of the physical world yet imbued with the power of imagination and creativity. It is this duality that defines us as human beings, this eternal struggle between the forces of light and darkness that drives us ever forward on our journey through life. We are beings of contradiction, capable of both great love and great cruelty, of profound wisdom and profound ignorance. And yet, it is precisely this paradox that makes us who we are, that gives meaning and purpose to our existence in an otherwise indifferent universe. In the end, it is our capacity for empathy and compassion that sets us apart from the rest of creation, that allows us to forge connections with one another and find solace in the embrace of community. For it is through our relationships with others that we truly come to know ourselves, to understand the depths of our own humanity and the boundless potential of the human spirit. And so, as we journey forth into the unknown, let us remember that we are not alone, that we are bound together by the ties of kinship and shared experience. In the final analysis, it is our shared humanity that defines us, that unites us in our common struggle for meaning and purpose in an uncertain world. And though the road ahead may be long and fraught with challenges, let us take comfort in the knowledge that we do not walk it alone, that we are surrounded by fellow travelers who share in our hopes and dreams. For it is together, hand in hand, that we will navigate the tumultuous waters of existence and find our way to the shores of a brighter tomorrow."]

import time

UAE-Large-V1 #2


In [2]:
from angle_emb import AnglE

angle = AnglE.from_pretrained('WhereIsAI/UAE-Large-V1', pooling_strategy='cls').cuda()

/Users/jiaweihong/Desktop/mmqg/.venv/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [3]:
start = time.time()
vec = angle.encode(sentence, to_numpy=True)
end = time.time()
print("Time taken: ", end - start)

Time taken:  1.455122947692871


SFR-Embedding-Mistral #1

In [4]:
import torch
import torch.nn.functional as F
from torch import Tensor
from transformers import AutoTokenizer, AutoModel

def last_token_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    left_padding = (attention_mask[:, -1].sum() == attention_mask.shape[0])
    if left_padding:
        return last_hidden_states[:, -1]
    else:
        sequence_lengths = attention_mask.sum(dim=1) - 1
        batch_size = last_hidden_states.shape[0]
        return last_hidden_states[torch.arange(batch_size, device=last_hidden_states.device), sequence_lengths]

def get_detailed_instruct(task_description: str, query: str) -> str:
    return f'Instruct: {task_description}\nQuery: {query}'

# load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained('Salesforce/SFR-Embedding-Mistral')
model = AutoModel.from_pretrained('Salesforce/SFR-Embedding-Mistral')

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
start_time = time.time()
  
# Embedding
batch_dict = tokenizer(sentence, max_length=512, padding=True, truncation=True, return_tensors="pt")
outputs = model(**batch_dict)
embeddings = last_token_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

end = time.time()
print("Time taken: ", end - start_time)

Time taken:  197.7794201374054



bge-large-en-v1.5 #3


In [6]:
from transformers import AutoTokenizer, AutoModel
import torch

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-large-zh-v1.5')
model = AutoModel.from_pretrained('BAAI/bge-large-zh-v1.5')
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(21128, 1024, padding_idx=0)
    (position_embeddings): Embedding(512, 1024)
    (token_type_embeddings): Embedding(2, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-23): 24 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inpl

In [8]:
start_time = time.time()
# Tokenize sentences
encoded_input = tokenizer(sentence, padding=True, truncation=True, max_length=512, return_tensors='pt')
# for s2p(short query to long passage) retrieval task, add an instruction to query (not add instruction for passages)
# encoded_input = tokenizer([instruction + q for q in queries], padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)
    # Perform pooling. In this case, cls pooling.
    sentence_embeddings = model_output[0][:, 0]
end = time.time()
print("Time taken: ", end - start_time)

Time taken:  5.155472755432129


GIST Large Embedding v0 #4

In [9]:
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer

revision = None  # Replace with the specific revision to ensure reproducibility in  case the model is updated.

model = SentenceTransformer("avsolatorio/GIST-large-Embedding-v0", revision=revision)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: avsolatorio/GIST-large-Embedding-v0


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps


In [10]:
start_time = time.time()

# Compute embeddings
embeddings = model.encode(sentence, convert_to_tensor=True)

end = time.time()
print("Time taken: ", end - start_time)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Time taken:  0.4793388843536377


bge-base-en-v1.5 #5


In [11]:
from transformers import AutoTokenizer, AutoModel
import torch

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-base-zh-v1.5')
model = AutoModel.from_pretrained('BAAI/bge-base-zh-v1.5')
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(21128, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [12]:
start_time = time.time()
# Tokenize sentences
encoded_input = tokenizer(sentence, padding=True, truncation=True, return_tensors='pt')
# for s2p(short query to long passage) retrieval task, add an instruction to query (not add instruction for passages)
# encoded_input = tokenizer([instruction + q for q in queries], padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)
    # Perform pooling. In this case, cls pooling.
    sentence_embeddings = model_output[0][:, 0]
end = time.time()
print("Time taken: ", end - start_time)

Time taken:  1.337890863418579
